In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import random
import numpy as np
np.random.seed(0)
random.seed(0)


/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_train2 = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [3]:
df_all = pd.concat([df_train, df_test],axis = 0)


In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64570 entries, 0 to 5270
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              64570 non-null  float64
 1   customer_country         63588 non-null  object 
 2   business_unit            64570 non-null  object 
 3   com_reg_ver_win_rate     16356 non-null  float64
 4   customer_idx             64570 non-null  int64  
 5   customer_type            19152 non-null  object 
 6   enterprise               64570 non-null  object 
 7   historical_existing_cnt  15031 non-null  float64
 8   id_strategic_ver         4037 non-null   float64
 9   it_strategic_ver         1174 non-null   float64
 10  idit_strategic_ver       5211 non-null   float64
 11  customer_job             44398 non-null  object 
 12  lead_desc_length         64570 non-null  int64  
 13  inquiry_type             62337 non-null  object 
 14  product_category         433

In [5]:

df_all["customer_country"] = df_all["customer_country"].str.split('/').str[-1].str.lower().str.strip()
filtered_countrys = df_all["customer_country"].value_counts()[df_all["customer_country"].value_counts() >= 30]

def filter_country(country):
    if pd.isna(country):
        return "none"
    elif country in filtered_countrys.index:
        return country
    else:
        return "else"
df_all["customer_country"] = df_all["customer_country"].apply(filter_country)


In [6]:
filtered_expected_timeline = df_all["expected_timeline"].str.replace(' ', '').str.replace('_', '').str.replace('.', '').value_counts()[df_all["expected_timeline"].str.replace(' ', '').str.replace('_', '').str.replace('.', '').value_counts() >= 100]

def filter_expected_timeline(expected_timeline):
    if pd.isna(expected_timeline):
            return "none" 
    elif expected_timeline in filtered_expected_timeline.index:
        return expected_timeline
    else:
        return "etc"

df_all["expected_timeline"] = df_all["expected_timeline"].str.replace(' ', '').str.replace('_', '').str.replace('.', '')
df_all["expected_timeline"] = df_all["expected_timeline"].apply(filter_expected_timeline)    

In [7]:
def categorize_inquiry(value):
    if pd.isna(value):
        return "none"
    else :  # 문자열인 경우에만 처리
        value = str(value).lower()
        if "quotation" in value or "purchase" in value:
            return "quotation or purchase"
        elif "sales" in value:
            return "sales"
        elif "information" in value or "trainings" in value:
            return "information"
        elif "technic" in value or "usage" in value:
            return "technical"
        else:
            return "else"
            
    
df_all["inquiry_type"] = df_all["inquiry_type"].apply(categorize_inquiry)

In [8]:
df_all["customer_type"] = df_all["customer_type"].str.replace(' ', '').str.replace('/', '').str.replace('_', '').str.lower().str.replace('-', '')
filtered_customer_type = df_all["customer_type"].value_counts()[df_all["customer_type"].value_counts() >= 30]

def filter_customer_type(customer_type):
    if pd.isna(customer_type):
        return "none"
    elif customer_type in filtered_customer_type.index:
        return customer_type
    else:
        return "others."
filtered_product_subcategory = df_all["product_subcategory"].value_counts()[df_all["product_subcategory"].value_counts() >= 20]

def filter_product_subcategory(product_subcategory):
    if pd.isna(product_subcategory):
        return "none"
    elif product_subcategory in filtered_product_subcategory.index:
        return product_subcategory
    else:
        return "etc."
    
df_all["product_subcategory"] = df_all["product_subcategory"].apply(filter_product_subcategory)
df_all["customer_type"] = df_all["customer_type"].apply(filter_customer_type)

In [9]:
    
filtered_product_category = df_all["product_category"].value_counts()[df_all["product_category"].value_counts() >= 30]

def filter_product_category(category):
    if pd.isna(category):
        return "none"
    elif category in filtered_product_category.index:
        return category
    else:
        return "etc."
df_all["product_category"] = df_all["product_category"].apply(filter_product_category)

In [10]:
filtered_jobs = df_all["customer_job"].value_counts()[df_all["customer_job"].value_counts() >= 20]

def filter_jobs(job):
    if pd.isna(job):
        return "none"
    elif job in filtered_jobs.index:
        return job
    else:
        return "else"
df_all["customer_job"] = df_all["customer_job"].apply(filter_jobs).replace({"others": "else", "other": "else"})  

In [11]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64570 entries, 0 to 5270
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              64570 non-null  float64
 1   customer_country         64570 non-null  object 
 2   business_unit            64570 non-null  object 
 3   com_reg_ver_win_rate     16356 non-null  float64
 4   customer_idx             64570 non-null  int64  
 5   customer_type            64570 non-null  object 
 6   enterprise               64570 non-null  object 
 7   historical_existing_cnt  15031 non-null  float64
 8   id_strategic_ver         4037 non-null   float64
 9   it_strategic_ver         1174 non-null   float64
 10  idit_strategic_ver       5211 non-null   float64
 11  customer_job             64570 non-null  object 
 12  lead_desc_length         64570 non-null  int64  
 13  inquiry_type             64570 non-null  object 
 14  product_category         645

In [12]:
df_train = df_all.iloc[: len(df_train)]
df_test = df_all.iloc[len(df_train) :]

In [13]:
df=df_train

In [14]:
filtered_df = df[(df['business_unit'].notna()) & (df['is_converted'] == True)]


In [15]:
bu_df=filtered_df['business_unit']

In [16]:
filtered_df2 = df[(df['business_unit'].notna()) & (df['is_converted'] == False)]


In [17]:
bu_df2=filtered_df2['business_unit']

In [18]:
nan_fd2=filtered_df2.dropna(axis=1)

In [19]:
nan_fd=filtered_df.dropna(axis=1)

In [20]:
d_F = nan_fd2.sample(n=4850, random_state=42) 

In [21]:
d_T=nan_fd

In [22]:
F_D_F= pd.concat([d_T, d_F], axis=0)


In [23]:
F_D_F

,bant_submit,customer_country,business_unit,customer_idx,customer_type,enterprise,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,lead_owner,is_converted
0,1.00,philippines,AS,32160,endcustomer,Enterprise,purchasing,62,quotation or purchase,multi-split,none,entry level,LGEPH,lessthan3months,1,0,0,1
1,1.00,philippines,AS,23122,endcustomer,Enterprise,media and communication,96,quotation or purchase,multi-split,none,ceo/founder,LGEPH,lessthan3months,1,0,1,1
2,1.00,india,AS,1755,endcustomer,Enterprise,engineering,56,information,single-split,none,partner,LGEIL,lessthan3months,1,0,2,1
3,1.00,india,AS,4919,endcustomer,Enterprise,entrepreneurship,44,quotation or purchase,vrf,none,ceo/founder,LGEIL,lessthan3months,1,0,3,1
4,1.00,india,AS,17126,specifierinfluencer,Enterprise,consulting,97,quotation or purchase,multi-split,none,partner,LGEIL,lessthan3months,0,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,0.50,india,ID,24197,none,SMB,engineering,107,quotation or purchase,standard signage,UHD TV Signage,associate/analyst,LGEIL,none,0,1,175,0
52512,0.75,france,Solution,34091,none,SMB,media and communication,145,quotation or purchase,"monitor signage,monior/monitor tv",none,ceo/founder,LGEKR,3months~6months,0,0,441,0
25249,0.75,philippines,AS,20394,endcustomer,Enterprise,marketing,94,quotation or purchase,single-split,none,manager,LGEPH,none,0,0,121,0
12756,0.75,hong kong,ID,20720,channelpartner,SMB,sales,33,quotation or purchase,high brightness signage,etc.,none,LGEHK,lessthan3months,0,0,337,0


In [24]:
filtered_customer_position = F_D_F["customer_position"].value_counts()[F_D_F["customer_position"].value_counts() >= 20]

def filter_customer_position(customer_position):
    if customer_position in filtered_customer_position.index:
        return customer_position
    else:
        return "others."

In [25]:
filtered_customer_position

customer_position
none                 3231
manager              1506
ceo/founder          1326
other                 971
director              798
associate/analyst     523
partner               458
entry level           275
c-level executive     144
others                103
trainee                88
vice president         81
intern                 59
end-user               44
hospital               21
Name: count, dtype: int64

In [26]:

F_D_F["customer_country"] = F_D_F["customer_country"].apply(filter_country)

In [27]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [28]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "business_unit",
    "enterprise",
    "customer_position",
    "response_corporate",
    "customer_country",
    "product_category",
    "inquiry_type",
    "expected_timeline",
    "customer_job",
    "product_subcategory",
    "customer_type"
]

df_all = pd.concat([F_D_F[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [29]:
for col in label_columns:  
    F_D_F[col] = df_all.iloc[: len(F_D_F)][col]
    df_test[col] = df_all.iloc[len(F_D_F) :][col]

In [30]:
train_data, test_data = train_test_split(F_D_F, test_size=0.2, random_state=42)

In [31]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier(random_state = 0)

y_train = train_data["is_converted"]
X_train = train_data.drop(["is_converted"],axis = 1)
y_test = test_data["is_converted"]
X_test = test_data.drop(["is_converted"],axis = 1)
model_xgb.fit(X_train, y_train)


y_xgb = model_xgb.predict(X_test)


acc_xgb = round(accuracy_score(y_test, y_xgb),4)*100

print("\n\nAccuracy with untuned XGBoost is: " + str(acc_xgb) + " %")



Accuracy with untuned XGBoost is: 92.73 %


In [32]:
from optuna.visualization import plot_edf
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

In [33]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import lightgbm as lgb
sampler = optuna.samplers.TPESampler(seed=0)  # TPESampler의 경우

def objective(trial):
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_test, label=y_test, reference=dtrain)

    param = {
        "seed": 0,
        "verbosity": -1,
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart", "goss"]),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    # GOSS를 사용할 때는 bagging 관련 파라미터를 제외
    if param["boosting_type"] != "goss":
        param["bagging_fraction"] = trial.suggest_float("bagging_fraction", 0.2, 1.0)
        param["bagging_freq"] = trial.suggest_int("bagging_freq", 1, 7)

    if param["boosting_type"] == "goss":
        top_rate = trial.suggest_float("top_rate", 0.0, 0.5)  # Adjust range if needed
        other_rate_max = 1.0 - top_rate
        other_rate = trial.suggest_float("other_rate", 0.0, other_rate_max)
        param["top_rate"] = top_rate
        param["other_rate"] = other_rate
        # Safety check to ensure sum does not exceed 1.0
        if param["top_rate"] + param["other_rate"] > 1.0:
            param["other_rate"] = 1.0 - param["top_rate"]
    if param["boosting_type"] == "dart":
        param["drop_rate"] = trial.suggest_float("drop_rate", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    callbacks = [
        lgb.early_stopping(stopping_rounds=100, verbose=False)
    ]

    # Train the model with the specified parameters, dataset, and callbacks
    gbm = lgb.train(
        param,
        dtrain,
        valid_sets=[dvalid],
        callbacks=callbacks,
        num_boost_round=1000  # Specify a large enough number to allow early stopping to occur
    )

    # Make predictions and evaluate
    preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize",sampler=sampler)
    study.optimize(objective, n_trials=500, timeout=6000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    # 최적의 하이퍼파라미터를 사용하여 모델을 학습
    best_params = trial.params
    # LightGBM의 Classifier를 사용합니다.
    best_lgbm = lgb.LGBMClassifier(**best_params,random_state=0)
    best_lgbm.fit(X_train, y_train, eval_set=[(X_test, y_test)])


[I 2024-02-26 03:42:40,012] A new study created in memory with name: no-name-bd69505d-7324-4df1-8069-3dcf7194aaeb
/home/elicer/.local/lib/python3.10/site-packages/lightgbm/callback.py:292: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')
[I 2024-02-26 03:42:47,363] Trial 0 finished with value: 0.9195876288659793 and parameters: {'boosting_type': 'dart', 'lambda_l1': 0.0008015832747965142, 'lambda_l2': 6.499698237449666e-05, 'num_leaves': 166, 'feature_fraction': 0.550069769010154, 'min_child_samples': 90, 'bagging_fraction': 0.9709302084008236, 'bagging_freq': 3, 'drop_rate': 0.021567524728780586, 'skip_drop': 0.0001702783205227671}. Best is trial 0 with value: 0.9195876288659793.
/home/elicer/.local/lib/python3.10/site-packages/lightgbm/callback.py:292: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')
[I 2024-02-26 03:42:49,330] Trial 1 finish

Number of finished trials:  500
Best trial:
  Value: 0.9448453608247422
  Params: 
    boosting_type: dart
    lambda_l1: 5.2611849174054616e-05
    lambda_l2: 3.172421376349247
    num_leaves: 75
    feature_fraction: 0.3406819250529085
    min_child_samples: 57
    bagging_fraction: 0.9524987262758106
    bagging_freq: 4
    drop_rate: 2.7377882078834642e-06
    skip_drop: 5.206631832283015e-07
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.3406819250529085, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3406819250529085
[LightGBM] [Warning] lambda_l2 is set=3.172421376349247, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.172421376349247
[LightGBM] [Warning] lambda_l1 is set=5.2611849174054616e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.2611849174054616e-05
[LightGBM] [Warning] bagging_fraction is set=0.9524987262758106, 

In [34]:
best_lgbm

LGBMClassifier(bagging_fraction=0.9524987262758106, bagging_freq=4,
               boosting_type='dart', drop_rate=2.7377882078834642e-06,
               feature_fraction=0.3406819250529085,
               lambda_l1=5.2611849174054616e-05, lambda_l2=3.172421376349247,
               min_child_samples=57, num_leaves=75, random_state=0,
               skip_drop=5.206631832283015e-07)

In [35]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
sampler = optuna.samplers.TPESampler(seed=0)  # TPESampler의 경우
def objective(trial):
    bootstrap_type = trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS'])
    boosting_type = trial.suggest_categorical('boosting_type', ['Ordered', 'Plain'])
    
    # 'Ordered' boosting_type은 'SymmetricTree' grow_policy와 호환됩니다.
    if boosting_type == 'Ordered':
        grow_policy = 'SymmetricTree'
    else:
        grow_policy = trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])

    param = {
        'iterations': trial.suggest_int('iterations', 50, 300),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10.0),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 0.01, 1.0),
        'loss_function': 'Logloss',
        'verbose': False,
        'boosting_type': boosting_type,
        'grow_policy': grow_policy,
        'bootstrap_type': bootstrap_type
    }
    
    # Bayesian 부트스트랩이 선택된 경우 bagging_temperature 파라미터를 추가하고 subsample을 제외합니다.
    if bootstrap_type == 'Bayesian':
        param['bagging_temperature'] = trial.suggest_uniform('bagging_temperature', 0.0, 1.0)
    else:
        param['subsample'] = trial.suggest_uniform('subsample', 0.1, 1)

    cat_cls = CatBoostClassifier(**param,random_seed=0)
    
    cat_cls.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=False)
    
    preds = cat_cls.predict(X_test)
    # CatBoost의 predict 결과는 불리언 타입으로 변환할 필요가 없으므로, 해당 라인을 제거합니다.
    pred_labels = preds
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction='maximize',sampler=sampler)
    study.optimize(objective, n_trials=500, timeout=6000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # 최적의 하이퍼파라미터를 사용하여 모델을 학습
    best_params = trial.params
    best_cat = CatBoostClassifier(**best_params, loss_function='Logloss', verbose=False,random_seed=0)
    best_cat.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=300)
    



[I 2024-02-26 03:53:42,927] A new study created in memory with name: no-name-3b99d011-6020-4602-851c-844a3569bb99
/tmp/ipykernel_306284/3755808542.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.3),
/tmp/ipykernel_306284/3755808542.py:25: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10.0),
/tmp/ipykernel_306284/3755808542.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'scale_pos_weight': trial.suggest_uniform('sc

Number of finished trials:  500
Best trial:
  Value: 0.9402061855670103
  Params: 
    bootstrap_type: MVS
    boosting_type: Plain
    grow_policy: Depthwise
    iterations: 164
    depth: 7
    learning_rate: 0.18549993576237597
    random_strength: 78
    border_count: 231
    l2_leaf_reg: 1.0269986840832468e-08
    scale_pos_weight: 0.6827375360483046
    subsample: 0.9681596378634871


In [36]:
import numpy as np
import optuna

import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb


def objective(trial):
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)

    param = {
        "seed": 0,
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize",sampler=sampler)
    study.optimize(objective, n_trials=500, timeout=6000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
        # 최적의 하이퍼파라미터를 사용하여 모델을 학습
        best_params = trial.params
        best_model = xgb.XGBClassifier(**best_params, loss_function='Logloss', verbose=False,random_state=0)
        best_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=300)

[I 2024-02-26 04:01:16,415] A new study created in memory with name: no-name-3440a5b6-e7b0-4d12-baa5-8e78ffbe891f
[I 2024-02-26 04:01:16,456] Trial 0 finished with value: 0.4984536082474227 and parameters: {'booster': 'dart', 'lambda': 0.005175238826679791, 'alpha': 0.9789850336895127, 'subsample': 0.31955864372639503, 'colsample_bytree': 0.8945008458945714, 'max_depth': 3, 'min_child_weight': 7, 'eta': 9.78497820827326e-08, 'gamma': 0.06082271952794911, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0059764707147453786, 'skip_drop': 0.08532152464454834}. Best is trial 0 with value: 0.4984536082474227.
[I 2024-02-26 04:01:16,485] Trial 1 finished with value: 0.4984536082474227 and parameters: {'booster': 'gbtree', 'lambda': 7.582711205996161e-06, 'alpha': 0.006917107089291241, 'subsample': 0.33730374180915246, 'colsample_bytree': 0.6168292849633035, 'max_depth': 3, 'min_child_weight': 3, 'eta': 1.4066121216700767e-08, 'gamma': 0.022365655

Number of finished trials:  500
Best trial:
  Value: 0.9278350515463918
  Params: 
    booster: gbtree
[0]	validation_0-logloss:0.43760
[1]	validation_0-logloss:0.34763
[2]	validation_0-logloss:0.29262


/home/elicer/.local/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-logloss:0.27019
[4]	validation_0-logloss:0.24384
[5]	validation_0-logloss:0.22729
[6]	validation_0-logloss:0.21944
[7]	validation_0-logloss:0.21176
[8]	validation_0-logloss:0.20178
[9]	validation_0-logloss:0.19885
[10]	validation_0-logloss:0.19869
[11]	validation_0-logloss:0.19985
[12]	validation_0-logloss:0.19872
[13]	validation_0-logloss:0.19896
[14]	validation_0-logloss:0.19560
[15]	validation_0-logloss:0.19614
[16]	validation_0-logloss:0.19594
[17]	validation_0-logloss:0.19511
[18]	validation_0-logloss:0.19646
[19]	validation_0-logloss:0.19293
[20]	validation_0-logloss:0.19189
[21]	validation_0-logloss:0.19122
[22]	validation_0-logloss:0.19072
[23]	validation_0-logloss:0.18958
[24]	validation_0-logloss:0.19123
[25]	validation_0-logloss:0.19095
[26]	validation_0-logloss:0.19036
[27]	validation_0-logloss:0.19113
[28]	validation_0-logloss:0.19313
[29]	validation_0-logloss:0.19231
[30]	validation_0-logloss:0.19199
[31]	validation_0-logloss:0.19280
[32]	validation_0-log

/home/elicer/.local/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

[63]	validation_0-logloss:0.18838
[64]	validation_0-logloss:0.18917
[65]	validation_0-logloss:0.19038
[66]	validation_0-logloss:0.19098
[67]	validation_0-logloss:0.19107
[68]	validation_0-logloss:0.19110
[69]	validation_0-logloss:0.19071
[70]	validation_0-logloss:0.19139
[71]	validation_0-logloss:0.19054
[72]	validation_0-logloss:0.19070
[73]	validation_0-logloss:0.19178
[74]	validation_0-logloss:0.19242
[75]	validation_0-logloss:0.19214
[76]	validation_0-logloss:0.19326
[77]	validation_0-logloss:0.19435
[78]	validation_0-logloss:0.19480
[79]	validation_0-logloss:0.19538
[80]	validation_0-logloss:0.19526
[81]	validation_0-logloss:0.19577
[82]	validation_0-logloss:0.19684
[83]	validation_0-logloss:0.19705
[84]	validation_0-logloss:0.19709
[85]	validation_0-logloss:0.19620
[86]	validation_0-logloss:0.19597
[87]	validation_0-logloss:0.19563
[88]	validation_0-logloss:0.19554
[89]	validation_0-logloss:0.19647
[90]	validation_0-logloss:0.19902
[91]	validation_0-logloss:0.19832
[92]	validatio

/home/elicer/.local/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[62]	validation_0-logloss:0.18915
[63]	validation_0-logloss:0.18838
[64]	validation_0-logloss:0.18917
[65]	validation_0-logloss:0.19038
[66]	validation_0-logloss:0.19098
[67]	validation_0-logloss:0.19107
[68]	validation_0-logloss:0.19110
[69]	validation_0-logloss:0.19071
[70]	validation_0-logloss:0.19139
[71]	validation_0-logloss:0.19054
[72]	validation_0-logloss:0.19070
[73]	validation_0-logloss:0.19178
[74]	validation_0-logloss:0.19242
[75]	validation_0-logloss:0.19214
[76]	validation_0-logloss:0.19326
[77]	validation_0-logloss:0.19435
[78]	validation_0-logloss:0.19480
[79]	validation_0-logloss:0.19538
[80]	validation_0-logloss:0.19526
[81]	validation_0-logloss:0.19577
[82]	validation_0-logloss:0.19684
[83]	validation_0-logloss:0.19705
[84]	validation_0-logloss:0.19709
[85]	validation_0-logloss:0.19620
[86]	validation_0-logloss:0.19597
[87]	validation_0-logloss:0.19563
[88]	validation_0-logloss:0.19554
[89]	validation_0-logloss:0.19647
[90]	validation_0-logloss:0.19902
[91]	validatio

/home/elicer/.local/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[62]	validation_0-logloss:0.18915
[63]	validation_0-logloss:0.18838
[64]	validation_0-logloss:0.18917
[65]	validation_0-logloss:0.19038
[66]	validation_0-logloss:0.19098
[67]	validation_0-logloss:0.19107
[68]	validation_0-logloss:0.19110
[69]	validation_0-logloss:0.19071
[70]	validation_0-logloss:0.19139
[71]	validation_0-logloss:0.19054
[72]	validation_0-logloss:0.19070
[73]	validation_0-logloss:0.19178
[74]	validation_0-logloss:0.19242
[75]	validation_0-logloss:0.19214
[76]	validation_0-logloss:0.19326
[77]	validation_0-logloss:0.19435
[78]	validation_0-logloss:0.19480
[79]	validation_0-logloss:0.19538
[80]	validation_0-logloss:0.19526
[81]	validation_0-logloss:0.19577
[82]	validation_0-logloss:0.19684
[83]	validation_0-logloss:0.19705
[84]	validation_0-logloss:0.19709
[85]	validation_0-logloss:0.19620
[86]	validation_0-logloss:0.19597
[87]	validation_0-logloss:0.19563
[88]	validation_0-logloss:0.19554
[89]	validation_0-logloss:0.19647
[90]	validation_0-logloss:0.19902
[91]	validatio

In [37]:
X_train.columns

Index(['bant_submit', 'customer_country', 'business_unit', 'customer_idx',
       'customer_type', 'enterprise', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'customer_position', 'response_corporate', 'expected_timeline',
       'ver_cus', 'ver_pro', 'lead_owner'],
      dtype='object')

In [38]:
df_test[X_train.columns]

,bant_submit,customer_country,business_unit,customer_idx,customer_type,enterprise,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,lead_owner
0,0.00,9,1,47466,2,0,8,3,2,27,96,44,40,7,1,0,278
1,0.25,68,2,5405,2,1,28,23,3,27,96,44,46,7,0,0,437
2,1.00,66,1,13597,8,1,20,138,3,12,95,39,17,4,0,0,874
3,0.50,68,1,17204,4,0,40,3,3,27,18,44,46,6,0,0,194
4,1.00,9,1,2329,2,0,12,107,3,18,32,48,40,4,1,1,167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,0.50,9,0,40292,4,0,40,7,3,27,96,4,40,7,0,0,97
5267,0.25,68,2,47466,4,0,28,157,3,27,96,44,46,7,0,0,438
5268,0.75,9,0,46227,8,0,13,39,2,26,96,39,40,4,0,0,97
5269,0.00,23,2,45667,2,1,28,3,2,5,96,44,11,7,0,0,429


In [39]:
df_test

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,id
0,0.00,9,1,0.073248,47466,2,0,53.0,NaN,NaN,...,7,1,0,0.001183,0.049840,retail,Electronics & Telco,278,1,19844.0
1,0.25,68,2,NaN,5405,2,1,NaN,NaN,NaN,...,7,0,0,0.000013,NaN,transportation,Others,437,1,9738.0
2,1.00,66,1,NaN,13597,8,1,NaN,NaN,NaN,...,4,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,1,8491.0
3,0.50,68,1,0.118644,17204,4,0,NaN,NaN,NaN,...,6,0,0,0.001183,0.049840,retail,NaN,194,0,19895.0
4,1.00,9,1,0.074949,2329,2,0,2.0,1.0,NaN,...,4,1,1,0.003079,0.064566,corporate / office,Engineering,167,1,10465.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,0.50,9,0,NaN,40292,4,0,10.0,NaN,NaN,...,7,0,0,NaN,NaN,NaN,NaN,97,0,13855.0
5267,0.25,68,2,NaN,47466,4,0,0.0,NaN,NaN,...,7,0,0,NaN,NaN,NaN,NaN,438,1,7979.0
5268,0.75,9,0,NaN,46227,8,0,NaN,NaN,NaN,...,4,0,0,NaN,NaN,NaN,NaN,97,0,12887.0
5269,0.00,23,2,NaN,45667,2,1,NaN,NaN,NaN,...,7,0,0,NaN,NaN,NaN,NaN,429,1,17530.0


In [40]:

# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted","id"], axis=1)


lgbm_pred = best_lgbm.predict(df_test[X_train.columns].fillna(0))
lgbm_pred = lgbm_pred.astype('int')
test_pred = best_model.predict(df_test[X_train.columns].fillna(0))
test_pred_cat = best_cat.predict(df_test[X_train.columns].fillna(0))
test_pred_cat = test_pred_cat.astype('int')
sum(test_pred)  # True로 예측된 개수

ensemble_pred = (test_pred + test_pred_cat + lgbm_pred ) /3

# 확률을 바탕으로 최종 클래스 레이블을 결정합니다. 예를 들어, 0.5를 기준으로 합니다.
final_pred = (ensemble_pred >= 0.5).astype(int)

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")

# 예측 결과를 submission에 반영
df_sub["is_converted"] = final_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.3406819250529085, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3406819250529085
[LightGBM] [Warning] lambda_l2 is set=3.172421376349247, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.172421376349247
[LightGBM] [Warning] lambda_l1 is set=5.2611849174054616e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.2611849174054616e-05
[LightGBM] [Warning] bagging_fraction is set=0.9524987262758106, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9524987262758106


In [41]:
sum(final_pred)

1978

In [42]:
test_pred_cat

array([1, 1, 0, ..., 0, 0, 1])

In [43]:
df_sub['is_converted'].value_counts()

is_converted
0    3293
1    1978
Name: count, dtype: int64